<a href="https://colab.research.google.com/github/Anbarasu707/student_project/blob/master/skinpredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

tf.logging.set_verbosity( tf.logging.ERROR )

print( tf.VERSION )


1.14.0


In [0]:
import requests, zipfile, io

r = requests.get( 'https://github.com/shubham0204/Dataset_Archives/blob/master/mel_detect_processed.zip?raw=true' ) 
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()


In [0]:
DIMEN = 32

X1 = np.load( 'processed_data/x1.npy')
X2 = np.load( 'processed_data/x2.npy')
Y = np.load( 'processed_data/y.npy')

X1 = X1.reshape( ( X1.shape[0]  , DIMEN**2 * 3  ) ).astype( np.float32 )
X2 = X2.reshape( ( X2.shape[0]  , DIMEN**2 * 3  ) ).astype( np.float32 )

train_X1 = X1[ : 4500 ] 
train_X2 = X2[ : 4500 ] 
train_Y = Y[ : 4500 ] 

test_X1 = X1[ 4500 : ]
test_X2 = X2[ 4500 : ] 
test_Y = Y[ 4500 : ]
 
print(  train_X1.shape )
print(  train_X2.shape )
print(  train_Y.shape )

print(test_X1.shape )
print(test_X1.shape )
print(test_Y.shape) 


(4500, 3072)
(4500, 3072)
(4500, 1)
(541, 3072)
(541, 3072)
(541, 1)


In [0]:
import tensorflow.keras.backend as K

input_shape = ( (DIMEN**2) * 3 , )
convolution_shape = ( DIMEN , DIMEN , 3 )

kernel_size_1 = ( 8 , 8 )
kernel_size_2 = ( 6 , 6 )
kernel_size_3 = ( 4 , 4 )

pool_size_1 = ( 6 , 6 )
pool_size_2 = ( 4 , 4 )

strides = 1

seq_conv_model = [

	tf.keras.layers.Reshape( input_shape=input_shape , target_shape=convolution_shape),
	
	tf.keras.layers.Conv2D( 32, kernel_size=kernel_size_1 , strides=strides ,activation='relu' ),
	tf.keras.layers.MaxPooling2D(pool_size=pool_size_1, strides=strides ),
	
	tf.keras.layers.Conv2D( 64, kernel_size=kernel_size_2 , strides=strides ,activation='relu'),
	tf.keras.layers.MaxPooling2D(pool_size=pool_size_2 , strides=strides),
    
    tf.keras.layers.Conv2D( 128, kernel_size=kernel_size_3 , strides=strides ,activation='relu'),
    
	tf.keras.layers.Flatten(),
	
	tf.keras.layers.Dense( 3076 , activation=tf.keras.activations.sigmoid )

]

seq_model = tf.keras.Sequential( seq_conv_model )

input_x1 = tf.keras.layers.Input( shape=input_shape )
input_x2 = tf.keras.layers.Input( shape=input_shape )

output_x1 = seq_model( input_x1 )
output_x2 = seq_model( input_x2 )

distance_euclid = tf.keras.layers.Lambda( lambda tensors : K.abs( tensors[0] - tensors[1] ))( [output_x1 , output_x2] )
outputs = tf.keras.layers.Dense( 1 , activation=tf.keras.activations.sigmoid) ( distance_euclid )
model = tf.keras.models.Model( [ input_x1 , input_x2 ] , outputs )

model.compile( loss=tf.keras.losses.binary_crossentropy , optimizer=tf.keras.optimizers.Adam(lr=0.0001) , metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3072)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 3072)]       0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 3076)         32106212    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 3076)         0           sequential[1][0]             

In [0]:
model.fit( [ train_X1 , train_X2 ] , train_Y , epochs=5 , batch_size=100 ) 

Epoch 1/5
4500/4500 [==============================] - 101s 22ms/sample - loss: 0.6879 - acc: 0.5253
Epoch 2/5
4500/4500 [==============================] - 99s 22ms/sample - loss: 0.6297 - acc: 0.6449
Epoch 3/5
4500/4500 [==============================] - 100s 22ms/sample - loss: 0.4979 - acc: 0.7724
Epoch 4/5
4500/4500 [==============================] - 100s 22ms/sample - loss: 0.3346 - acc: 0.8791
Epoch 5/5
4500/4500 [==============================] - 99s 22ms/sample - loss: 0.1894 - acc: 0.9527


In [0]:
metrics = model.evaluate( [ test_X1 , test_X2 ] , test_Y ) 
print( 'Loss of {} and Accuracy is {} %'.format( metrics[0] , metrics[1] * 100 ) ) 


541/541 [==============================] - 3s 6ms/sample - loss: 0.2475 - acc: 0.9094
Loss of 0.24747702264521346 and Accuracy is 90.94269871711731 %


In [0]:
model.save( 'model.h5' ) 
converter = tf.lite.TFLiteConverter.from_keras_model_file( 'model.h5' ) 
converter.post_training_quantize = True
tflite_model = converter.convert()
size = open( 'recog_model.tflite' , 'wb' ).write( tflite_model ) 
print( 'Model size > {} Megabytes'.format( size / 1024**2))


/usr/local/lib/python3.6/dist-packages/tensorflow/lite/python/lite.py:769: UserWarning: Property post_training_quantize is deprecated, please use optimizations=[Optimize.DEFAULT] instead.
  " instead." % name)


Model size > 30.635719299316406 Megabytes
